In [1]:
import uproot
import glob
import pandas as pd
from tqdm import tqdm

import numpy as np
import awkward as ak
import ROOT
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.26/04


In [2]:
username = "yian"
import getpass
import os
if os.system('klist | grep Default | grep ' + username + '@CERN.CH'):
    os.system('echo %s' % getpass.getpass() + " | kinit " + username)

Default principal: yian@CERN.CH


In [3]:
year="18"
root_sig = glob.glob("/eos/user/y/yian/AJJ_analysis/add_weight/chain_combine/cutjet-EWAjj*"+year+".root")
root_bkg = glob.glob("/eos/user/y/yian/AJJ_analysis/add_weight/chain_combine/cutjet-GJets150ToInf*"+year+".root")

In [4]:
print(root_bkg,root_sig)

['/eos/user/y/yian/AJJ_analysis/add_weight/chain_combine/cutjet-GJets150ToInf_18.root'] ['/eos/user/y/yian/AJJ_analysis/add_weight/chain_combine/cutjet-EWAjjLO_18.root', '/eos/user/y/yian/AJJ_analysis/add_weight/chain_combine/cutjet-EWAjj_18.root']


In [5]:
branch = uproot.open(root_sig[1]+":Events").keys()
necessary_columns=[]
unnecessary_columns=['vjj_jets','vjj_mus','vjj_eles','vjj_photons','vjj_loosePhotons']
for i in branch:
    if ('vjj_' in i and 'gen' not in i and i not in unnecessary_columns) or ('eff' in i) or ('scalef' in i):        
        necessary_columns.append(i)

In [6]:
signal = uproot.lazy(root_sig[1]+':Events')[necessary_columns]
sig=pd.DataFrame(signal[necessary_columns].to_numpy()).query("vjj_isGood==1 & vjj_jj_m>500 & vjj_v_pt>200 & vjj_lead_pt>50 & vjj_sublead_pt>50 & vjj_trig>0")

In [8]:
background=uproot.lazy(root_bkg[0]+':Events')[necessary_columns]
bkg=pd.DataFrame(background[necessary_columns].to_numpy()).query("vjj_isGood==1 & vjj_jj_m>500 & vjj_v_pt>200 & vjj_lead_pt>50 & vjj_sublead_pt>50 & vjj_trig>0")

In [9]:
sig['isSignal']=1
bkg['isSignal']=0

In [ ]:
features=['vjj_jj_m','vjj_v_ystar','vjj_jj_dphi','vjj_vjj_scalarht','vjj_lead_qgl','vjj_vjj_dphi','vjj_vjj_circularity','vjj_sublead_qgl','vjj_jj_pt','vjj_vjj_D','vjj_vjj_C','vjj_sublead_pt','vjj_jj_deta','vjj_vjj_aplanarity']

In [ ]:
df = pd.concat([sig,bkg],ignore_index=True)

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
X=df[features]
Y=df['isSignal']
    #random_state is the seed used by the random number generator, 
    #random_state=42 make others get the same data spliting in different executions.
    # shuffle make the bkg and signal mix randomly.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, 
                                                        random_state=42,shuffle=True)

In [ ]:
xgbc = XGBClassifier(eval_metric = "auc").fit(X_train, y_train)
xgbc.save_model("model.json")

model=xgb.Booster()
model.load_model("./model.json")

In [ ]:
y=pd.DataFrame(y_test,columns=['isSignal'])

test_score_xgb = xgbc.predict_proba(X_test)
test_score_xgb_sig = test_score_xgb[:,1]

dtest=xgb.DMatrix(X_test)
y['score']=model.predict(dtest)

plt.hist(y.query('isSignal==1')['score'],label = "Sig",histtype="step",bins = np.linspace(0,1),density=True)
plt.hist(y.query('isSignal==0')['score'],label = "Bkg",histtype="step",bins = np.linspace(0,1),density=True)
plt.legend()
plt.xlim(0,1)
plt.savefig('separation.pdf')

In [ ]:
import sklearn.metrics as m
from sklearn.metrics import roc_curve, auc
fpr_xgb, tpr_xgb, _thres = m.roc_curve(y_test,test_score_xgb[:,1])

roc_auc = auc(fpr_xgb, tpr_xgb)

f, ax = plt.subplots(figsize=(8, 8))
#ax.plot(np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), '--', label='random')
ax.plot(fpr_xgb, tpr_xgb, label='XGBoost Classifier (area = %0.2f)'% roc_auc)

#ax.set_yscale('log');
ax.set_xlim(0, 1); ax.set_ylim(1e-3, 1)
ax.set_xlabel('Backgroun efficiency', ha='right', x=1.0); ax.set_ylabel('Signal efficiency', ha='right', y=1.0)
ax.legend()
plt.savefig('ROC.pdf')

In [ ]:
from xgboost import plot_importance
plt.figure(figsize = (10,10))
ax = plt.subplot(111)
plot_importance(xgbc,ax=ax)
plt.savefig('Feature_importance.pdf')

In [ ]:
score_sig=xgbc.predict_proba(sig[features])[:,1]
score_bkg=xgbc.predict_proba(bkg[features])[:,1]
sig['score']=score_sig
bkg['score']=score_bkg

In [ ]:
plt.hist(bkg['score'],label = "BKG",histtype="step",bins = np.linspace(0,1),density=False,weights=bkg['scalef']*bkg['vjj_photon_effWgt']*59.7)
plt.hist(sig['score'],label = "SIG",histtype="step",bins = np.linspace(0,1),density=False,weights=sig['scalef']*sig['vjj_photon_effWgt']*59.7)
plt.yscale('symlog')
plt.ylabel('Events')
plt.legend()
plt.xlim(-0.1,1.1)
#plt.savefig('sig_bkg_distribution.pdf')

In [ ]:
dsig=xgb.DMatrix(sig[features])
dbkg=xgb.DMatrix(bkg[features])
sig['score1']=model.predict(dsig)
bkg['score1']=model.predict(dbkg)

In [ ]:
plt.hist(bkg['score1'],label = "BKG",histtype="step",bins = np.linspace(0,1),density=False)
plt.hist(sig['score1'], label = "SIG",histtype="step",bins = np.linspace(0,1),density=False)
plt.yscale('symlog')
plt.ylabel('Events')
plt.legend()
plt.xlim(-0.1,1.1)

In [ ]:
plt.hist(bkg['score1'],label = "BKG",histtype="step",bins = np.linspace(0,1),density=False,weights=bkg['scalef']*bkg['vjj_photon_effWgt']*59.7)
plt.hist(sig['score1'], label = "SIG",histtype="step",bins = np.linspace(0,1),density=False,weights=sig['scalef']*sig['vjj_photon_effWgt']*59.7)
plt.yscale('symlog')
plt.ylabel('Events')
plt.legend()
plt.xlim(-0.1,1.1)